!TODO:
    
- Explore more complex nor unique preprocessing (singkatan, tidak baku, bahasa inggris, pos tagging)

# Import modules & overview


In [1]:
# basic import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import math

%matplotlib inline

In [2]:
import spacy
import re

In [3]:
link_train = "./dataset/train.csv"
link_test = "./dataset/test.csv"

In [4]:
train_df = pd.read_csv(link_train, delimiter=";")
test_df = pd.read_csv(link_test, delimiter=";")

In [5]:
# train_df['text'] = train_df['stemmed_text']
# train_df.drop(columns=['label_angka', 'stemmed_text', 'Unnamed: 0'], inplace=True)

In [6]:
train_df.head()

,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik


In [7]:
test_df.head()

,IDText,Text
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan i...
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib Lah ...


Mari kita eksplor dataset, mulai dari ukuran, informasi dan detail kolom.

In [8]:
train_df.nunique()

text     4583
label       8
dtype: int64

In [9]:
train_df['label'].value_counts()

label
Politik                    2972
Sosial Budaya               587
Pertahanan dan Keamanan     400
Ideologi                    400
Ekonomi                     367
Sumber Daya Alam            192
Demografi                    62
Geografi                     20
Name: count, dtype: int64

# Text Preprocessing

In [10]:
def counter_word(df, view=10):
    counter = df['text'].apply(lambda x:str(x).split())
    top = Counter([item for sublist in counter for item in sublist])
    temp = pd.DataFrame(top.most_common(view))
    temp.columns = ['Common_words','count']
    return temp.style.background_gradient(cmap='Blues')

def counter_word2(df, view=10, length=5):
    counter = df['text'].apply(lambda x:str(x).split())
    top = Counter([item for sublist in counter for item in sublist if len(item) > length])
    temp = pd.DataFrame(top.most_common(view))
    temp.columns = ['Common_words','count']
    return temp.style.background_gradient(cmap='Blues')

In [11]:
counter_word2(train_df, 30, 11)

,Common_words,count
0,#GanjarPranowoPilihanUmat,658
1,#GanjarMahfudRebound,558
2,#GanjarMahfud2024,377
3,Ganjar-Mahfud,190
4,#AMINAjaDulu,172
5,Prabowo-Gibran,172
6,#02Melanjutkan,143
7,Yurissa_Samosir],103
8,#DuluJokowiSekarangGanjar,100
9,Mdy_Asmara1701],96


In [41]:
process_df = train_df.copy()

## Drop duplicated texts

Data yang duplikat hanya memperberat teks untuk di preproses dan model untuk belajar, belum juga risiko model cenderung bias akibat data duplikat.

In [42]:
process_df.duplicated().sum()

381

In [43]:
process_df.drop_duplicates(inplace=True)

In [44]:
process_df.duplicated().sum()

0

In [45]:
process_df.shape

(4619, 2)

From 5000 texts turned into 4609 texts fr training and validation

## Removing Unecessary words

Yang termasuk sebagai kata yang tidak penting diantaranya username twitter atau mention, kata yang terlalu panjang

Penjelasan regex:

- @ = Mencocokkan simbol @ sebagai awalan username.
- [A-Za-z0-9+/=]+ =  Mencocokkan satu atau lebih karakter huruf (besar/kecil), angka, tanda plus (+), garis miring (/), atau sama dengan (=).
- (?=\s|$$) =  Menjamin bahwa pola diakhiri dengan spasi (\s) atau akhir baris ($), tanpa meninggalkan karakter setelah username.

In [46]:
# hapus mention @
process_df['text'] = process_df['text'].str.replace(r'@[A-Za-z0-9+/=]+(?=\s|$)', '', regex=True)
process_df.head(10)

,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik
2,"emng bener sih, pendukung 01 ada yg goblok, b...",Demografi
3,RT Sewaktu anies bersikap kritis ke kinerja p...,Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik
5,RT Duh jangan sampai Pak lurah denger nih Ã°ÂŸ...,Politik
6,Prabowo minta Kemenhan dan Tim Satgas Air Unha...,Pertahanan dan Keamanan
7,RT Anies: Ya Allah Sibukkanlah Orang Zalim Aga...,Politik
8,"RT Abah itu bapa kami, bapa yg berikan perhat...",Politik
9,"RT Bawaslu dimaki ""JANCUK"" sama warga garaÃ‚Â²...",Politik


In [70]:
rows = 8
row = train_df.iloc[rows]  
row2 = process_df.iloc[rows] 
print(row['text']) 
print()
print(row2['text']) 

RT Abah @znOMP7ZMVU9dMuMNA/cIazC9q5+hgwVkTsQNdQLKgTc= itu bapa kami, bapa yg berikan perhatian untuk pendidikan anak anak Papua jadi pintar pintar. Gerakan Indonesia mengajar tak bisa di lupakan sama warga Papua. Kami ingin bapa Anies jadi Presiden. https://t.co/FCdf6J1L3Z [RE LANGKAHANIES]

 abah bapa bapa yg berikan perhatian pendidikan anak anak papua jadi pintar pintar gerakan indonesia mengajar tak di lupakan sama warga papua ingin bapa anies jadi presiden re langkahanies


In [55]:
def counter_word_username(df, view=10):
    counter = df['text'].apply(lambda x:str(x).split())
    top = Counter([item for sublist in counter for item in sublist if '@' in item])
    temp = pd.DataFrame(top.most_common(view))
    temp.columns = ['Common_words','count']
    return temp.style.background_gradient(cmap='Blues')

# Top 20 username yang TAG dalam teks training
counter_word_username(process_df, 100)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

Kita bisa melihat bahwa semua username telah bersih, yang tersisa dapat kita hilangkan dengan dictionary sederhana

Penjelasan regex: 

- \[: Mencocokkan tanda kurung siku pembuka [.
- [^\]]+: Mencocokkan satu atau lebih karakter apapun kecuali tanda kurung siku penutup ].
- \]: Mencocokkan tanda kurung siku penutup ].

In [56]:
# hapus username RE
process_df['text'] = process_df['text'].str.replace(r'\[[^\]]+\]', '', regex=True)

In [31]:
def counter_word_username2(df, view=10):
    counter = df['text'].dropna().apply(lambda x: re.findall(r'\[RE ([^\]]+)\]', x)) 
    top = Counter([item for sublist in counter for item in sublist])
    temp = pd.DataFrame(top.most_common(view), columns=['Username', 'Count'])
    return temp.style.background_gradient(cmap='Blues')

counter_word_username2(process_df, 10)

,Username,Count


In [48]:
# hapus link
process_df['text'] = process_df['text'].str.replace(r'https?://\S+', '', regex=True)

In [49]:
# hapus angka dan tanda baca
process_df['text'] = process_df['text'].str.replace(r'[\d!"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~]', '', regex=True)

In [50]:
# hapus kata yang typo lebih dari 20 kata
process_df['text'] = process_df['text'].str.replace(r'\b\w{20,}\b', '', regex=True)

In [51]:
# Menghapus semua karakter non-alfabet dan non-spasi
process_df['text'] = process_df['text'].str.replace(r'[^A-Za-z\s]', '', regex=True)

In [52]:
# Menghapus kata "RT"
process_df['text'] = process_df['text'].str.replace(r'\bRT\b', '', regex=True)

# Mengganti spasi berlebihan dengan satu spasi
process_df['text'] = process_df['text'].str.replace(r'\s{2,}', ' ', regex=True)

In [53]:
process_df.drop_duplicates(inplace=True) # memastikan tidak ada duplikat

Berikut merupakan bruteforce regex filtering kata-kata yang setidaknya tidak digunakan

## Case folding

In [57]:
process_df['text'] = process_df['text'].str.lower()

In [58]:
process_df

,text,label
0,kunjungan prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,anies dapat tepuk tangan meriah saat jadi rek...,Politik
2,emng bener sih pendukung ada yg goblok begitu...,Demografi
3,sewaktu anies bersikap kritis ke kinerja pak ...,Politik
4,anies baswedan harap asn termasuk tni dan polr...,Politik
...,...,...
4995,ngeliat debat kemaren pas prabowo kicep kekir...,Politik
4996,masyarakat yakin bahwa prabowogibran memiliki ...,Politik
4997,imo both are irrational but yg satu jauh lebih...,Ekonomi
4998,look at that pak ganjar anda sdh berkecimpung...,Pertahanan dan Keamanan


In [69]:
print(train_df.loc[0, 'text'])
print()
print(process_df.loc[8, 'text'])

Kunjungan Prabowo ini untuk meresmikan dan menyerahkan proyek bantuan air bersih di lima titik. #IndonesiaSentris #IndonesiaHijau #02Melanjutkan #AnakMudaIndonesiaEmas Prabowo Subianto

 abah bapa bapa yg berikan perhatian pendidikan anak anak papua jadi pintar pintar gerakan indonesia mengajar tak di lupakan sama warga papua ingin bapa anies jadi presiden re langkahanies


## Stopping Stopword

### Sastrawi

Kami menggunakan library Sastrawi untuk stopping stopword, stemming teks Bahasa Indonesia

In [64]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [65]:
factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

print(stopwords)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [66]:
stopwords = factory.create_stop_word_remover()

In [67]:
process_df['text'] = process_df['text'].apply(stopwords.remove)

In [68]:
# process_df.duplicated().sum()

### Twitter stopwords

https://github.com/Braincore-id/IndoTWEEST

In [141]:
stopwords_df = pd.read_csv('./dictionary/stopwords_twitter.csv')  # Ganti dengan path ke file Anda
stopwords = set(stopwords_df['33books'].str.strip().tolist())  # Mengambil daftar stopword

# Membuat regex untuk mencocokkan stopword
stopword_pattern = r'\b(?:' + '|'.join(map(re.escape, stopwords)) + r')\b'

# Menghapus stopword dari teks
process_df['text'] = process_df['text'].str.replace(stopword_pattern, '', regex=True)

# Menghapus spasi berlebihan (jika diperlukan)
process_df['text'] = process_df['text'].str.replace(r'\s{2,}', ' ', regex=True).str.strip()

### Other stopwords handling

https://github.com/nasalsabila/kamus-alay

In [142]:
slang_df = pd.read_csv('./dictionary/colloquial-indonesian-lexicon.csv')  # Ganti dengan path ke file CSV Anda
slang_dict = dict(zip(slang_df['slang'], slang_df['formal']))  # Membuat dictionary slang -> formal

# Membuat regex pattern untuk semua slang
slang_pattern = r'\b(?:' + '|'.join(map(re.escape, slang_dict.keys())) + r')\b'

# Fungsi untuk mengganti slang dengan formal
def replace_slang(text):
    return re.sub(slang_pattern, lambda match: slang_dict[match.group(0)], text)

# Menerapkan fungsi ke kolom teks
process_df['text'] = process_df['text'].apply(replace_slang)

In [147]:
# process_df.duplicated().sum()
process_df.drop_duplicates(inplace=True) # memastikan tidak ada duplikat

## Stemming text

In [148]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/pityudhistira28/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/pityudhistira28/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pityudhistira28/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [149]:
df = process_df.copy()

In [150]:
def tokenize_text(sentence):
    token = word_tokenize(sentence)
    return token

In [151]:
df['token'] = df['text'].apply(tokenize_text)
df.head()

,text,label,token
0,kunjungan prabowo untuk meresmikan menyerahkan...,Sumber Daya Alam,"[kunjungan, prabowo, untuk, meresmikan, menyer..."
1,anies tepuk tangan meriah jadi rektor mewajibk...,Politik,"[anies, tepuk, tangan, meriah, jadi, rektor, m..."
2,benar pendukung goblok pendukung hanya saja pa...,Demografi,"[benar, pendukung, goblok, pendukung, hanya, s..."
3,sewaktu anies bersikap kritis kinerja pak prab...,Politik,"[sewaktu, anies, bersikap, kritis, kinerja, pa..."
4,anies baswedan harap asn termasuk tni polri pe...,Politik,"[anies, baswedan, harap, asn, termasuk, tni, p..."


In [152]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

In [153]:
def stemming_text(tokens):
    result = [stemmer.stem(token) for token in tokens]
    return result

In [154]:
df['stemmed'] = df['token'].apply(stemming_text)

In [155]:
df['stemmed_text'] = df['stemmed'].apply(lambda tokens: ' '.join(tokens))
df

,text,label,token,stemmed,stemmed_text
0,kunjungan prabowo untuk meresmikan menyerahkan...,Sumber Daya Alam,"[kunjungan, prabowo, untuk, meresmikan, menyer...","[kunjung, prabowo, untuk, resmi, serah, proyek...",kunjung prabowo untuk resmi serah proyek bantu...
1,anies tepuk tangan meriah jadi rektor mewajibk...,Politik,"[anies, tepuk, tangan, meriah, jadi, rektor, m...","[anies, tepuk, tangan, riah, jadi, rektor, waj...",anies tepuk tangan riah jadi rektor wajib mata...
2,benar pendukung goblok pendukung hanya saja pa...,Demografi,"[benar, pendukung, goblok, pendukung, hanya, s...","[benar, dukung, goblok, dukung, hanya, saja, p...",benar dukung goblok dukung hanya saja pak ridw...
3,sewaktu anies bersikap kritis kinerja pak prab...,Politik,"[sewaktu, anies, bersikap, kritis, kinerja, pa...","[waktu, anies, sikap, kritis, kerja, pak, prab...",waktu anies sikap kritis kerja pak prabowo ang...
4,anies baswedan harap asn termasuk tni polri pe...,Politik,"[anies, baswedan, harap, asn, termasuk, tni, p...","[anies, baswedan, harap, asn, masuk, tni, polr...",anies baswedan harap asn masuk tni polri pegan...
...,...,...,...,...,...
4995,melihat debat kemarin pas prabowo kicep kekira...,Politik,"[melihat, debat, kemarin, pas, prabowo, kicep,...","[lihat, debat, kemarin, pas, prabowo, kicep, k...",lihat debat kemarin pas prabowo kicep kira kel...
4996,masyarakat yakin prabowogibran memiliki visi s...,Politik,"[masyarakat, yakin, prabowogibran, memiliki, v...","[masyarakat, yakin, prabowogibran, milik, visi...",masyarakat yakin prabowogibran milik visi jala...
4997,imo both are irrational but satu jauh lebih ir...,Ekonomi,"[imo, both, are, irrational, but, satu, jauh, ...","[imo, both, are, irrational, but, satu, jauh, ...",imo both are irrational but satu jauh lebih ir...
4998,look that pak ganjar sudah berkecimpung lgisla...,Pertahanan dan Keamanan,"[look, that, pak, ganjar, sudah, berkecimpung,...","[look, that, pak, ganjar, sudah, kecimpung, lg...",look that pak ganjar sudah kecimpung lgislatif...


In [156]:
process_df['text'] = df['stemmed_text']

In [158]:
process_df.duplicated().sum()

0

# Export

In [159]:
process_df.to_csv('./dataset/clean.csv', index=False)